1. Capturando os dados




In [ ]:
#importando as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
#lendo o arquivo que baixei da investing
df_base_original = pd.read_csv('/content/dados_ibovespa1.csv')

In [ ]:
#conferindo que os dados subiram no DF
df_base_original.head()

In [ ]:
#analisando os tipos de dados
df_base_original.info()

2.   Preparação dos dados



In [ ]:
#convertendo a data para datetime
df_base_original['Data'] = pd.to_datetime(df_base_original['Data'], format='%d.%m.%Y')

#Limpando e convertendo 'Vol.' para float, removendo a letra 'M' e K, convertendo para milhões e milhares
df_base_original['Vol.'] = df_base_original['Vol.'].replace({'M': 'e6', 'K': 'e3'}, regex=True).replace(',', '.', regex=True).astype(float)

# Convertendo o 'Var%' para float, removendo o símbolo '%' e dividindo por 100
df_base_original['Var%'] = df_base_original['Var%'].str.replace('%', '').str.replace(',', '.').astype(float) / 100

In [ ]:
# Removendo espaços e caracteres especiais dos nomes das colunas
df_base_original.columns = df_base_original.columns.str.strip().str.lower().str.replace('%', '').str.replace('.', '').str.replace(' ', '_')

In [ ]:
#instalando uma biblioteca para tratativa da acentuação do nome das colunas, espaços existentes e deixando tudo em letra minuscula
!pip install unidecode

In [ ]:
import unidecode
df_base_original.columns = [unidecode.unidecode(col).strip().lower() for col in df_base_original.columns]

In [ ]:
# Exportando o DataFrame para um arquivo Excel para analisar o estado completo do arquivo
df_base_original.to_excel('df_base_tratada.xlsx', index=False)

In [ ]:
# Preencher valores ausentes na coluna 'vol' com a média, descobri que há uma linha com o dado vazio
df_base_original['vol'].fillna(df_base_original['vol'].mean(), inplace=True)

In [ ]:
#revalidando através de função que deu certo e não há dados em branco
df_base_original.isnull().sum()

In [ ]:
#verificação das mudanças
print(df_base_original.columns)


3. Explorando os dados

In [ ]:
# Ordenação por data
df_base_original = df_base_original.sort_values('data')

# Plote da série temporal do fechamento diário
plt.figure(figsize=(14, 7))
plt.plot(df_base_original['data'], df_base_original['ultimo'], label='Fechamento')
plt.title('Série Temporal do Fechamento Diário do IBOVESPA')
plt.xlabel('Data')
plt.ylabel('Fechamento (R$)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Configurações do estilo para o grafico com indicadores de queda
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12, 8))

# Linha do fechamento
plt.plot(df_base_original['data'], df_base_original['ultimo'], color='lightblue', label='Fechamento')

# Calculo das quedas significativas, maior que 5% - circuit break
df_base_original['var_percentual'] = df_base_original['ultimo'].pct_change() * 100
df_base_original['queda_significativa'] = df_base_original['var_percentual'] < -5

# Adicionando áreas sombreadas para quedas significativas
plt.fill_between(df_base_original['data'], df_base_original['ultimo'],
                 where=df_base_original['queda_significativa'], color='red', alpha=0.3,
                 label='Quedas Significativas (>5%)')

# Média móvel para suavizar os dados
df_base_original['media_movel'] = df_base_original['ultimo'].rolling(window=30).mean()
plt.plot(df_base_original['data'], df_base_original['media_movel'], color='blue', linestyle='--',
         label='Média Móvel (30 dias)')

# Personalização do gráfico
plt.title('Histórico de Fechamento do IBOVESPA (2004-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Data', fontsize=12)
plt.ylabel('Fechamento', fontsize=12)
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Decomposição da série temporal com base nas datas
result = sm.tsa.seasonal_decompose(df_base_original['ultimo'], model='additive', period=252)
plt.rcParams.update({'figure.figsize': (10, 10)})
result.plot()
plt.show()




4. Criação do modelo

In [ ]:
!pip install prophet

In [ ]:
import pandas as pd
from prophet import Prophet

#Dados exigidos para analise
df_prophet = df_base_original[['data', 'ultimo']].rename(columns={'data': 'ds', 'ultimo': 'y'})

# Filtrar dados até o final de 2018-02-22 que é o valor de 70% da minha base
df_filtered = df_prophet[df_prophet['ds'] < '2018-02-24']
train_data = df_filtered

# atribuindo os dados para teste a partir de 2018-02-22, os 30% restante
test_data = df_prophet[df_prophet['ds'] >= '2018-02-24']

# Exibindo a divisão
print(f"Tamanho do treino: {len(train_data)}")
print(f"Tamanho do teste: {len(test_data)}")


In [ ]:
#gerando o excel para conferir os dados
test_data.to_excel('test_data.xlsx', index=False)
train_data.to_excel('train_data.xlsx', index=False)

In [ ]:
#gerando uma visualização grafics dos dados de treino e teste
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plotar dados de treino
plt.plot(train_data['ds'], train_data['y'], label='Dados de Treino', color='blue')

# Plotar dados de teste
plt.plot(test_data['ds'], test_data['y'], label='Dados de Teste', color='orange')

plt.title('Comparação entre Dados de Treino e Dados de Teste')
plt.xlabel('Data')
plt.ylabel('Valor de Fechamento')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
from prophet import Prophet
from prophet.make_holidays import make_holidays_df

# Criando o modelo Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.3,
    seasonality_mode='additive',
    interval_width=0.95
)

# Extrair os anos que temos na base para criação do mapeamento de feriados usando a função make_holidays_df
years = df_prophet['ds'].dt.year.unique()

# Feriados estaduais e municipais
sp_holidays = make_holidays_df(
    year_list=years,
    country='BR', province='SP',
)

# Feriados nacionais
br_holidays = make_holidays_df(
    year_list=years,
    country='BR',
)

# Mensalão
mensalao = pd.DataFrame({
    'holiday': 'Escândalo do Mensalão',
    'ds': pd.to_datetime(['2005-01-01', '2005-06-01']),  # Datas representativas do escândalo
    'lower_window': 0,
    'upper_window': 0,
})

# Pré-sal
pre_sal = pd.DataFrame({
    'holiday': 'Descoberta Pré-sal',
    'ds': pd.to_datetime(['2006-01-01']),
    'lower_window': 0,
    'upper_window': 0,
})

# Crise Financeira 2008
crise_2008 = pd.DataFrame({
    'holiday': 'Crise Financeira 2008',
    'ds': pd.to_datetime(['2008-09-15']),
    'lower_window': -5,
    'upper_window': 0,
})

# Crise da Dívida da Europa
crise_divida_europa = pd.DataFrame({
    'holiday': 'Crise da Dívida da Europa',
    'ds': pd.to_datetime(['2012-01-01', '2012-06-01', '2012-11-01']),  # Datas representativas da crise
    'lower_window': 0,
    'upper_window': 0,
})

# Impeachment 2014
impeachment = pd.DataFrame({
    'holiday': 'Impeachment 2014',
    'ds': pd.to_datetime(['2014-03-01']),
    'lower_window': -5,
    'upper_window': 5,
})

# Eleições a cada 4 anos, a partir de 2002
anos_eleicoes = np.arange(2002, 2026, 4)
dia_mes = ['10-04'] * len(anos_eleicoes)
datas_eleicoes = [f'{ano}-{dia}' for ano, dia in zip(anos_eleicoes, dia_mes)]

eleicoes = pd.DataFrame({
    'holiday': 'Eleição Presidencial',
    'ds': pd.to_datetime(datas_eleicoes),
    'lower_window': -3,
    'upper_window': 4,
})

# Juntar as bases
holidays_df = pd.concat([sp_holidays, br_holidays, mensalao, pre_sal, crise_2008, impeachment, eleicoes, crise_divida_europa])
holidays_df.drop_duplicates(inplace=True)

# Usar atribuição direta para evitar warnings
holidays_df['upper_window'] = holidays_df['upper_window'].fillna(4)
holidays_df['lower_window'] = holidays_df['lower_window'].fillna(-3)

# Adicionar sazonalidades personalizadas
model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
model.add_seasonality(name='weekly', period=7, fourier_order=10)

# Treinar o modelo
model.fit(df_prophet)


In [ ]:
# colocando a previsão do mesmo tamanho que a base de testes
n_periods = len(test_data)
future = model.make_future_dataframe(periods=n_periods)

# Criar DataFrame futuro
forecast = model.predict(future)

# Filtrar as previsões até 2024
forecast = forecast[forecast['ds'] <= '2024-03-31']

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Previsões para os dados de teste
forecast_test = forecast[forecast['ds'].isin(test_data['ds'])]

# Pegando as previsões para as datas que estão tanto no forecast quanto no test_data
common_dates = forecast_test['ds'].isin(test_data['ds'])
y_true = test_data[test_data['ds'].isin(forecast_test['ds'])]['y'].values
y_pred = forecast_test[common_dates]['yhat'].values

# Verificando os tamanhos
print(f"Tamanho de y_true: {len(y_true)}")
print(f"Tamanho de y_pred: {len(y_pred)}")

# Calcular métricas se os tamanhos forem iguais
if len(y_true) == len(y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)

    print(f'MAE: {mae}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
else:
    print("Erro: os tamanhos de y_true e y_pred ainda não correspondem.")


In [ ]:
print("Duplicatas em test_data:")
print(test_data[test_data.duplicated(subset='ds')])

print("Valores nulos em forecast:")
print(forecast[forecast['yhat'].isnull()])


In [ ]:
# Verificando últimas 10 previsões
print(forecast[['ds', 'yhat']].tail(10))


In [ ]:
#Grafico das comparações
plt.figure(figsize=(10, 6))
plt.plot(test_data['ds'], test_data['y'], label='Real', color='blue')
plt.plot(forecast['ds'], forecast['yhat'], label='Previsto', color='red', linestyle='--')
plt.axvline(x=test_data['ds'].min(), color='gray', linestyle='--', label='Início do Teste')
plt.title('Comparação de Previsões')
plt.xlabel('Data')
plt.ylabel('Valores')
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
#grafico das comparações com foco nos dados previstos
import matplotlib.pyplot as plt
import pandas as pd

# Criar um DataFrame com os valores reais e as previsões
comparison_df = pd.DataFrame({
    'Data': forecast_test['ds'],
    'Real': y_true,
    'Previsão': y_pred
})

plt.figure(figsize=(10, 6))
plt.plot(comparison_df['Data'], comparison_df['Real'], label='Valor Real', color='blue')
plt.plot(comparison_df['Data'], comparison_df['Previsão'], label='Valor Previsto', color='red', linestyle='--')
plt.title('Comparação entre o Valor Real e o Valor Previsto do Ibovespa')
plt.xlabel('Data')
plt.ylabel('Valor de Fechamento')
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Validação cruzada
from prophet.diagnostics import cross_validation, performance_metrics

initial = '1825 days'
horizon = '150 days'
period = '150 days'

df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)

# Calcular as métricas de performance
df_p = performance_metrics(df_cv)

# Visualizar as primeiras linhas das métricas
print(df_p.head())

# Visualizar as métricas de desempenho
print("Métricas de Desempenho:")
print(f"Mean Absolute Error (MAE): {df_p['mae'].mean()}")
print(f"Root Mean Squared Error (RMSE): {df_p['rmse'].mean()}")
print(f"Mean Absolute Percentage Error (MAPE): {df_p['mape'].mean()}")
